In [1]:
# Sarah loves music (incorporating collaboration)

from neurotron import Neurotron,Monitor,Record,toy,cat

#===========================================================================
# class Cells
#===========================================================================

class Cells:
    def __init__(self,n,par,sizes,verbose=0):
        self.verbose = verbose
        self.cells = []
        for k in range(n):
            cell = Neurotron(k,par,sizes,'cell-%g'%k)
            self.cells.append(cell)
        self.record = Record(self)
        
    def __call__(self,y,log=None):
        if log is not None:
            if self.verbose > 0: print("\n"+log)
            log = None if self.verbose == 0 else log
        for k in range(len(self.cells)):
            y = self.cells[k](y,log)

        self.record(self)

        if (self.verbose > 0):
            self.log(y,log)
        return y

    def __getitem__(self,k):
        return self.cells[k]

    def range(self):
        return range(len(self.cells))
        
    def plot(self,j,tag=''):   # plot cell column at monitor location (*,j)
        for k in range(len(self.cells)):
            mon(self.cells[k],k,j)
        mon.xlabel(j,tag)

    def log(self,y,tag=None):
        self.record.log(self,y,tag)

    def iterate(self,k,y,tag,log=None):
        prefix = log + " - " if log is not None else ""
        y = self(y,'iteration %g (%s)'%(k,prefix+tag))
        self.plot(k,tag)
        return y
    
    def iteration(self,y,log=None):
        self.record.clear()
        tags = ['stimu','react','excite','burst','settle','relax1','relax2','relax3']
        for k in range(len(tags)):
            y = self.iterate(k,y,tags[k],log)
        return y
            
#===========================================================================
# helper: check cell's movie with given pattern
#===========================================================================

def check(cells,pattern):
    """
    check(): check recorder pattern of a network of cells against a given
             pattern and raise exception if check fails.
    >>> par,token = toy('sarah')
    >>> cells = Cells(3,par,[3,10])
    >>> y = cells.iteration([0,0,0]+token['Sarah'])
    >>> check(cells,'|U,U,QB,QBY,DY,D--|---D,XD,X,X,X|---D,D---|')
    """
    p = cells.record.pattern()
    if p != pattern:
        print('expected: ',pattern)
        print('got:      ',p)
        raise Exception('pattern check failed')



ImportError: cannot import name 'Neurotron' from 'neurotron' (/Users/hux/Bluenetics/Git/Neural/neurotron-develop/neurotron/src/neurotron/__init__.py)

In [ ]:
mon = Monitor(3,8,'Scenario 1: How to create a cell cluster')

#===========================================================================
# How to create a cells cluster 
#===========================================================================

par,token = toy('sarah')
cells = Cells(3,par,[3,10],verbose=0)
y = cells.iteration([0,0,0]+token['Sarah'])
check(cells,'|U,U,UQB,UQBY,DY,D,D-|---XDS,XD,XD,X,X|---D,D,D--|')

In [ ]:
#===========================================================================
# Secario 2: No predictive cells 
#===========================================================================
par,token = toy('sarah');  epar,cpar,ppar,dyn = par
epar.p[2] = epar.p[1] = epar.p[0]             # overwrite => all the same
ppar.P[2][0] = ppar.P[1][0] = ppar.P[0][0]    # overwrite => all the same

y,sizes = cat(c:=[0,0,0],token['Sarah'])
cells = Cells(3,par,sizes,verbose=0)

mon = Monitor(3,8,'Scenario 2: No Predictive Cells')
y = cells.iteration(y)
check(cells,'|U,U,UQB,UQBY,DY,D,D-|U,U,UQB,UQDBY,DY,D,D-|U,U,UQB,UQDBY,DY,D,D-|')

In [2]:
#===========================================================================
# Secario 3: some predictive cells 
#===========================================================================
par,token = toy('sarah');  epar,cpar,ppar,dyn = par
epar.p[2] = epar.p[1] = epar.p[0]             # overwrite => all the same
ppar.P[2][0] = ppar.P[1][0] = ppar.P[0][0]    # overwrite => all the same

y,sizes = cat(c:=[0,0,0],token['Sarah'])
cells = Cells(3,par,sizes,verbose=0)
cells[0].x.set(1)
cells[2].x.set(1)

mon = Monitor(3,8,'Scenario 3: Some Predictive Cells')
y = cells.iteration(y)
check(cells,'|UX,UXLY,UQXDY,UQXDY,XDY,XD,XD-|U,UD,UQD,UQD,D,D,D-|UX,UXLDY,UQXDY,UQXDY,XDY,XD,XD-|')

NameError: name 'toy' is not defined

In [3]:
#===========================================================================
# Secario 4: Sequence Prediction -> 'Sarah'
#===========================================================================
par,token = toy('sarah');  epar,cpar,ppar,dyn = par
#par[0].p[2] = par[0].p[1] = par[0].p[0] # overwrite => all the same
#ppar.delta = (0,0)                     # disable learning
cpar.p[0] = cpar.p[1] = cpar.p[2] = [0,0,0]   # no collaboration 

mon = Monitor(3,8,"Presentation of first item: 'Sarah'")
y,sizes = cat(c:=[0,0,0],token['Sarah'])
cells = Cells(3,par,sizes,verbose=0)
y = cells.iteration(y,'Sarah')
check(cells,'|U,U,UQB,UQBY,Y---|---XS,X,X,X,X|--------|')

mon = Monitor(3,8,"Presentation of second item: 'loves'")
y,sizes = cat(c:=y[:3],token['loves'])
y = cells.iteration(y,'loves')
check(cells,'|--------|UX,UXLY,UQXBY,UQXBY,Y---|-XS,X,X,X,X,X,X|')

mon = Monitor(3,8,"Presentation of third item: 'music'")
y,sizes = cat(c:=y[:3],token['music'])
y = cells.iteration(y,'music')
check(cells,'|--------|--------|UX,UXLY,UQXBY,UQXBY,Y---|')

NameError: name 'toy' is not defined